In [ ]:
import findspark

findspark.init()

In [ ]:
import pyspark

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [ ]:
from pyspark.ml.classification import LogisticRegression, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Load training data
data = spark.read.format("libsvm").load("iris-train.txt")

In [ ]:
# Split the training data into train and test for parameter tuning
splits = data.randomSplit([0.6, 0.4], 1)
train = splits[0]
test = splits[1]

# instantiate the base classifier.
lr = LogisticRegression(maxIter=10, tol=1E-6, fitIntercept=True)

# instantiate the One Vs Rest Classifier.
ovr = OneVsRest(classifier=lr)

# train the multiclass model.
ovrModel = ovr.fit(train)

In [ ]:
# compute accuracy on the test set
result = ovrModel.transform(test)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

In [ ]:
result.show(result.count(), truncate=25)